### GANS - GENERATIVE ADVERSARIAL NETWORKS

Suppose there is a shop which buys certain kinds of artwork for customers which they will later resell. However, there are nerfarious customers who sell **fake** artworks. In this case, the shop has to distinguish between the fake and authentic artworks.

You can imagine that initially, the forger might make a lot of mistakes when trying to sell the fake piece of art. The art inspector can easily find the fake from the authentic pieces. Because of this the forger tries recreating the work as close as possible to the real one.

So, the forger **learns to make the artwork more REAL as possible**

Meanwhile, the art inspector will also learn how new forgers are coming up with ideas to forge a piece of art. This can be through his friend circles in the art community. 

So, the art-inspector **learns to tell the authentic ones from the fake ones**

#### ARCHITECTURE OF GANS

The GANS has the following architecture

![image.png](attachment:image.png)

The main components of the GAN architecture are as follows:

- Noise Vector: This is the pallete from the Forger's inspirational mind. This is also known as the **Latent Sample**.

- Generator: This is the Forger who creates the fake artwork. This is a **Generative Network** which is typically a Convolutional Neural Network with **deconvolutional layers**. This network takes some noise vector as input and produces an image. The goal of the Generator is to produce images that are as real as possible.

- Discriminator: This is the Art Inspector who inspects the art work from the fake and real ones. This is a **Discriminator Network** which is typically a Convolutional Neural Network which assigns probability on the image on whether it's real or not. 

**ULTIMATE GOAL** - Discriminator must produce art works(outputs) that are as close as possible to the real ones fooling the inspector to say it's real always. 

### A SIMPLE GAN MODEL WITH KERAS

We are going to develop a simple GAN model using the MNIST Handwritten digits using the Python `Keras` library.  

In [1]:
# Importing all the libraries required
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

Using TensorFlow backend.


In [0]:
# To make sure that we can reproduce the experiment and get the same results
np.random.seed(10)

# The dimension of our random noise vector.
random_dim = 100

Before starting out on generating the images we must load the MNIST Handwritten digits into the library. This is done below using the Helper function `load_mnist_data()`.

In [0]:
def load_minst_data():
    # Load the MNIST data from the keras library
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    # Normalize the training data between [-1,1]
    x_train = (x_train.astype(np.float32)- 127.5)/ 127.5
    
    # Reshape the training data to (60000,784)-> a column vector
    x_train = x_train.reshape(60000,784)
    
    return(x_train,y_train,x_test,y_test)

### DESIGN THE OPTIMIZER, GENERATOR, DISCRIMINATOR

We are going to desing the Adam Optimizer which is based on Stochastic Gradient Descent. This is done with the helper function `get_optimizer()`. This is followed by the Generator and the Discriminator functions which are used to define the CNN layers for the model.

In [0]:
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

# Generator function
def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer =optimizer)
    
    return generator

# Discriminator Model
def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

In [0]:
# This is the Whole GAM Network
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

For complete visualizations, we can create another helper function `plot_generated_images()` which will save images that will save the generated image every 30 epochs.

In [0]:
# Create a wall of generated MNIST images
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [0]:
def train(epochs=1, batch_size=128):
    # Get the training and testing data
    x_train, y_train, x_test, y_test = load_minst_data()
    # Split the training data into batches of size 128
    batch_count = x_train.shape[0] / batch_size

    # Build our GAN netowrk
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(int(batch_count))):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # Generate fake MNIST images
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)

train(400, 128)

11493376/11490434 [==============================] - 1s 0us/step






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


  0%|          | 0/468 [00:00<?, ?it/s]

--------------- Epoch 1 ---------------










  1%|▏         | 6/468 [00:00<00:08, 56.38it/s]

--------------- Epoch 2 ---------------


...

  1%|▏         | 6/468 [00:00<00:08, 53.51it/s]

--------------- Epoch 223 ---------------


  1%|▏         | 6/468 [00:00<00:07, 59.74it/s]

--------------- Epoch 224 ---------------


 41%|████      | 192/468 [00:03<00:05, 54.59it/s]

### RESULTS:

The GAN was able to acheive reasonable results in 400 eopchs here are some image snapshots from the epochs 1, 100, 200 and 400
**EPOCH 1**
![Epoch-1](https://github.com/Lingesh2311/GANS/blob/master/gan_generated_image_epoch_1.png?raw=1)
**EPOCH 100**
![Epoch-1](https://github.com/Lingesh2311/GANS/blob/master/gan_generated_image_epoch_100.png?raw=1)
**EPOCH 200**
![Epoch-1](https://github.com/Lingesh2311/GANS/blob/master/gan_generated_image_epoch_200.png?raw=1)